<a href="https://colab.research.google.com/github/Ximena5745/Desempeno_Institucional/blob/main/Consolidado_Ind.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import glob
import pandas as pd

In [ ]:
!git clone https://github.com/Ximena5745/Desempeno_Institucional.git

In [ ]:
os.chdir("/content/repositorio/carpeta_excel")